<a href="https://colab.research.google.com/github/wingyiuc/COMP3359_AI_Ap_Project/blob/master/Data_processing_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Getting started

In [1]:
""" Prepare Notebook for Google Colab """
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify directory of course materials in Google Drive
module_dir = '/content/drive/My Drive/COMP3359 AI Applications Group Project'

# Add course material directory in Google Drive to system path, for importing .py files later
# (Ref.: https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab)
import sys
sys.path.append(module_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# !pip install fastparquet
!pip install ta

  Created wheel for ta: filename=ta-0.5.22-cp36-none-any.whl size=24340 sha256=57862797db3fb5e7bb65063e7f287b21c2461735e7a628378bf3a657259eaea1
  Stored in directory: /root/.cache/pip/wheels/87/0c/4d/646b17c066f52a8f6923b17ef23839a19584ee2b4d55238465
Successfully built ta


In [3]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import glob
import requests
import matplotlib
import math
import ta
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
raw_data_dir = os.path.join(module_dir, "Cryptocurrency data/Raw data/")
processed_data_dir = os.path.join(module_dir, "Cryptocurrency data/Processed data/")
# raw_data_dir = os.path.join(module_dir, "Downloaded data/Daily data/Raw data/")
# processed_data_dir = os.path.join(module_dir, "Downloaded data/Daily data/Processed data/")
os.listdir(raw_data_dir)

['BTC_price.csv',
 'ETH_price.csv',
 'DOGE_price.csv',
 'USDT_price.csv',
 'DASH_price.csv',
 'LTC_price.csv',
 'NMC_price.csv',
 'PPC_price.csv',
 'NVC_price.csv',
 'TRC_price.csv',
 'FRC_price.csv',
 'FTC_price.csv',
 'IXC_price.csv',
 'WDC_price.csv',
 'BTB_price.csv']

## 1. Feature Engineering

In [0]:
def FeatureEngineering(df, num_holding_days = 1):
  df['time'] = pd.to_datetime(df['time'])
  df = df.set_index('time')
  df = df[~df.index.duplicated()]
  df['ret_holding'] = (df['close'] / df['close'].shift(num_holding_days) - 1).shift(-num_holding_days)
  # df['15SMA'] = ta.trend.ema_indicator(df['close'],n=15)
  df['15SMA'] = ta.trend.sma_indicator(df['close'],n=15)
  df['SMA_ret_holding1'] = (df['15SMA'] / df['15SMA'].shift(1) - 1).shift(-1)
  # df['SMA_ret_holding1'] = (df['15SMA'] / df['close'])-1


  df['buy_signal'] = df['SMA_ret_holding1'].apply(lambda x: 1 if x > 0.005 else 0)
  df['sell_signal'] = df['SMA_ret_holding1'].apply(lambda x: 1 if -x > 0.005 else 0)
  df['hold_signal'] = np.where(df['buy_signal']==0,np.where(df["sell_signal"]==0,1,0),0)


  # df.drop(columns='ret_holding',inplace=True)
  df['close_ret'] = (df['close'] / df['close'].shift(num_holding_days) - 1)
  df['12EMA'] = ta.trend.ema_indicator(df['close'], 12)
  df['50EMA'] = ta.trend.ema_indicator(df['close'], 50)
  df['100EMA'] = ta.trend.ema_indicator(df['close'], 100)
  df['minusDI'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'],12).adx_neg()
  df['plusDI'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'],12).adx_pos()
  df['MACD'] = ta.trend.MACD(df['close']).macd_diff()
  # df['MACDsignal'] = ta.trend.MACD(df['close']).macd_signal()
  df['RSI'] = ta.momentum.RSIIndicator(df['close']).rsi()
  df['StoOsci'] = ta.momentum.StochasticOscillator(df['close'], df['high'], df['low']).stoch()
  df['StoOsci_signal'] = ta.momentum.StochasticOscillator(df['close'], df['high'], df['low']).stoch_signal()
  df['BB_hband'] = ta.volatility.BollingerBands(df['close'],n=14).bollinger_hband()
  df['BB_lband'] = ta.volatility.BollingerBands(df['close'],n=14).bollinger_lband()
  df['ATR'] = ta.volatility.AverageTrueRange(df['high'],df['low'],df['close']).average_true_range()
  df['ADI'] = ta.volume.AccDistIndexIndicator(df['high'],df['low'],df['close'],df['volumeto']).acc_dist_index()
  df['MFI'] = ta.volume.MFIIndicator(df['high'],df['low'],df['close'],df['volumeto']).money_flow_index()
  df = df.dropna()
  return df

In [6]:
ticker = 'BTC'
data_dir = glob.glob(raw_data_dir+ticker+"_price.csv")[0]
df = pd.read_csv(data_dir)
num_holding_days = 1
y = ['buy_signal', "sell_signal", "hold_signal"]

# Feature Engineering
df = FeatureEngineering(df, num_holding_days)
df.head()

,close,high,low,open,volumefrom,volumeto,ret_holding,15SMA,SMA_ret_holding1,buy_signal,sell_signal,hold_signal,close_ret,12EMA,50EMA,100EMA,minusDI,plusDI,MACD,RSI,StoOsci,StoOsci_signal,BB_hband,BB_lband,ATR,ADI,MFI
time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-04-11,124.90,188.7,110.20,165.0,118747.72,18867223.23,-0.063251,133.710000,0.015367,1,0,0,-0.243030,145.284333,87.712684,60.798513,19.204489,28.229613,2.618666,52.534220,12.516431,30.091575,213.472003,60.737997,30.593601,1.095791e+08,62.609170
2013-04-12,117.00,140.0,54.25,124.9,556468.03,47619189.45,-0.205128,135.764667,0.001228,0,0,1,-0.063251,140.932898,88.861206,61.911414,27.806557,22.955601,-1.440933,50.594625,-29.038179,-2.600867,211.884738,66.110976,34.533343,1.316533e+08,52.254324
2013-04-13,93.00,130.0,85.50,117.0,238347.58,26277415.26,-0.032258,135.931333,-0.001074,0,0,1,-0.205128,133.558606,89.023512,62.527030,25.146848,20.759888,-5.680621,45.141858,-6.093979,-7.538576,211.795017,66.316412,35.245247,1.142335e+08,47.746081
2013-04-14,90.00,110.0,84.44,93.0,166274.96,16197017.56,-0.084556,135.785333,-0.005224,0,1,0,-0.032258,126.857282,89.061805,63.071049,23.959408,19.586011,-8.457816,44.496306,-17.387097,-17.506418,212.140937,65.537634,34.553444,1.050830e+08,45.445031
2013-04-15,82.39,102.0,71.51,90.0,236673.01,20607774.11,-0.170288,135.076000,-0.017590,0,1,0,-0.084556,120.016161,88.800166,63.453602,25.223837,18.243578,-10.447686,42.823364,-23.820312,-15.767130,214.284307,60.307122,34.263198,9.918256e+07,41.923978


Inspecting the data

In [7]:
print('number of buy signals',df['buy_signal'].sum())
print('number of sell signals',df['sell_signal'].sum())
print('number of hold signals',df['hold_signal'].sum())

number of buy signals 826
number of sell signals 641
number of hold signals 1075


## 2. Preprocess all tickers

In [0]:
num_holding_days = 1
tickers = ['BTC', 'ETH', 'DOGE', 'USDT', 'DASH', "LTC", "NMC", "PPC", "NVC", "TRC", "FRC", "FTC", "IXC", "WDC", "BTB"]
for ticker in tickers:
  price_dir = glob.glob(raw_data_dir+ticker+"_price.csv")
  if len(price_dir)==0:
    print(ticker,' not found')
    continue
  else:
    price_dir = price_dir[0]
  df = pd.read_csv(price_dir)
  df = FeatureEngineering(df, num_holding_days)
  df.to_csv(processed_data_dir+ticker+'_processed.csv')

/usr/local/lib/python3.6/dist-packages/ta/trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  din[i+self._n] = 100 * (self._din[i]/self._trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:595: RuntimeWarning: invalid value encountered in double_scalars
  dip[i+self._n] = 100 * (self._dip[i]/self._trs[i])
